In [1]:
import socket, os
from pyspark.sql import SparkSession


spark_hostname = socket.gethostname()
spark_ip_address = socket.gethostbyname(spark_hostname)
minio_ip_address = "62.72.11.215"

## DEFINIDO O MESMO IP POIS OS SERVIÇOS VÃO EXECUTAR NO MESMO SERVIDOR
print(f"SPARK: {spark_hostname} - {spark_ip_address}")
print(f"MINIO: {minio_ip_address}")

spark = (
    SparkSession.builder
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.1,com.amazonaws:aws-java-sdk-bundle:1.12.469,org.apache.hadoop:hadoop-aws:3.3.4")

    # CONFIGURA EXTENSÃO DO DELTA
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

    # LIMITA USO DE CORE E MEMORIA POR EXECUTOR
    .config("spark.cores.max", "1")
    .config("spark.executor.memory", "2g")

    # CONFIGURAÇÃO PARA COMUNICAR COM PROTOCOLO S3
    .config("spark.driver.bindAddress", f"{spark_hostname}")
    .config("spark.driver.host", f"{spark_ip_address}")
    .config("spark.hadoop.fs.s3a.access.key", "minio")
    .config("spark.hadoop.fs.s3a.secret.key", "minio123")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.endpoint", f"http://{minio_ip_address}:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

    .config("spark.network.timeout", "480s")
    .config("spark.scheduler.barrier.maxConcurrentTasksCheck.maxFailures", "5")
    .config("spark.rdd.compress", "true")
    .config("spark.shuffle.compress", "true")
    .config("spark.shuffle.spill.compress", "true")
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:+G1SummarizeConcMark")
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC -XX:+G1SummarizeConcMark")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryoserializer.buffer.max", "512m")

    .config("spark.sql.autoBroadcastJoinThreshold", -1)

    # CRIA SESSÃO DO SPARK
    .master(f"spark://{spark_ip_address}:7077")
    .appName("rascunho")
    .getOrCreate()
)

# hadoop_base_path = os.getenv("HADOOP_HOME").replace("\\", "/")
# hadoop_config = spark.sparkContext._jsc.hadoopConfiguration()
# hadoop_config.set("driver.extraClassPath", f"{hadoop_base_path}/lib/native/hadoop-aws-3.3.1.jar:{hadoop_base_path}/lib/native/aws-java-sdk-1.12.153")

# for key, value in [(k.replace("spark.hadoop.", ""), v) for k, v in spark.sparkContext.getConf().getAll() if k.find("hadoop") != -1]:
#     hadoop_config.set(key, value)

SPARK: note_rns - 192.168.18.118
MINIO: 62.72.11.215


In [3]:
estados = spark.read.csv(
    path="s3a://datalake/raw/municipios_brasileiros/estados.csv.gz", 
    schema="codigo_uf LONG, uf STRING, nome STRING, latitude FLOAT, longitude FLOAT, regiao STRING",
    header=True
)

estados.printSchema()

root
 |-- codigo_uf: long (nullable = true)
 |-- uf: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- regiao: string (nullable = true)



In [4]:
estados.show(truncate=False)

+---------+---+-------------------+--------+---------+--------+
|codigo_uf|uf |nome               |latitude|longitude|regiao  |
+---------+---+-------------------+--------+---------+--------+
|11       |RO |Rondônia           |-10.83  |-63.34   |Norte   |
|12       |AC |Acre               |-8.77   |-70.55   |Norte   |
|13       |AM |Amazonas           |-3.47   |-65.1    |Norte   |
|14       |RR |Roraima            |1.99    |-61.33   |Norte   |
|15       |PA |Pará               |-3.79   |-52.48   |Norte   |
|16       |AP |Amapá              |1.41    |-51.77   |Norte   |
|17       |TO |Tocantins          |-9.46   |-48.26   |Norte   |
|21       |MA |Maranhão           |-5.42   |-45.44   |Nordeste|
|22       |PI |Piauí              |-6.6    |-42.28   |Nordeste|
|23       |CE |Ceará              |-5.2    |-39.53   |Nordeste|
|24       |RN |Rio Grande do Norte|-5.81   |-36.59   |Nordeste|
|25       |PB |Paraíba            |-7.28   |-36.72   |Nordeste|
|26       |PE |Pernambuco         |-8.38

In [3]:
municipios = spark.read.csv(
    path="s3a://datalake/raw/municipios_brasileiros/municipios.csv.gz", 
    schema="codigo_ibge LONG, nome STRING, latitude FLOAT, longitude FLOAT, capital INT, codigo_uf LONG",
    # schema="codigo_ibge LONG, nome STRING, latitude FLOAT, longitude FLOAT, capital INT, codigo_uf LONG, siafi_id LONG, ddd LONG, fuso_horario STRING",
    header=True
)

municipios.printSchema()

root
 |-- codigo_ibge: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- capital: integer (nullable = true)
 |-- codigo_uf: long (nullable = true)



In [4]:
municipios.show(truncate=False)

+-----------+-------------------+--------+---------+-------+---------+
|codigo_ibge|nome               |latitude|longitude|capital|codigo_uf|
+-----------+-------------------+--------+---------+-------+---------+
|5200050    |Abadia de Goiás    |-16.7573|-49.4412 |0      |52       |
|3100104    |Abadia dos Dourados|-18.4831|-47.3916 |0      |31       |
|5200100    |Abadiânia          |-16.197 |-48.7057 |0      |52       |
|3100203    |Abaeté             |-19.1551|-45.4444 |0      |31       |
|1500107    |Abaetetuba         |-1.72183|-48.8788 |0      |15       |
|2300101    |Abaiara            |-7.34588|-39.0416 |0      |23       |
|2900108    |Abaíra             |-13.2488|-41.6619 |0      |29       |
|2900207    |Abaré              |-8.72073|-39.1162 |0      |29       |
|4100103    |Abatiá             |-23.3049|-50.3133 |0      |41       |
|4200051    |Abdon Batista      |-27.6126|-51.0233 |0      |42       |
|1500131    |Abel Figueiredo    |-4.95333|-48.3933 |0      |15       |
|42001

# SALVE REFINED

In [23]:
from pyspark.sql.functions import current_timestamp

refined = (
    spark.read.load(
        path="s3a://datalake/trusted/municipios_brasileiros",
        format="delta",
    )
    .selectExpr([
        "codigo_ibge",
        "nome",
        "e_capital",
        "uf",
        "nome_estado",
        "regiao",
        "current_timestamp() AS gerado_em"
    ])
)

(
    refined.write
    .option("overwriteSchema", "true")
    .format("delta")
    .mode("overwrite")
    .save("s3a://datalake/refined/municipios_brasileiros")
)

# SALVE TRUSTED

In [6]:
trusted = (
    municipios.alias("m").join(
        other=estados.alias("e"),
        on=(municipios.codigo_uf == estados.codigo_uf),
        how="inner"
    )
    .selectExpr([
        "m.codigo_ibge",
        "m.nome",
        "m.latitude",
        "m.longitude",
        "(m.capital == 1) AS e_capital",
        "e.uf",
        "e.nome AS nome_estado",
        "e.regiao",
        "current_timestamp() AS gerado_em",
    ])
    .distinct()
)

# trusted.show(truncate=False)

trusted = (
    trusted
    .withMetadata("codigo_ibge", { "tipo": "NUMERICO", "descricao": "Código composto de 7 dígitos, sendo os dois primeiros referentes ao código da Unidade da Federação" })
    .withMetadata("nome", { "tipo": "TEXTO", "descricao": "Nome do município" })
    .withMetadata("latitude", { "tipo": "NUMERICO", "descricao": "Latitude do município" })
    .withMetadata("longitude", { "tipo": "NUMERICO", "descricao": "Longitude do município" })
    .withMetadata("e_capital", { "tipo": "BOOLEANO", "descricao": "Identifica se o município é ou não capital do estado" })
    .withMetadata("uf", { "tipo": "TEXTO", "descricao": "Unidade Federativa que identifica o estado" })
    .withMetadata("nome_estado", { "tipo": "TEXTO", "descricao": "Nome do estado" })
    .withMetadata("regiao", { "tipo": "TEXTO", "descricao": "Agrupamento das UFs" })
    .withMetadata("gerado_em", { "tipo": "DATA E HORA", "descricao": "Identifica a data e hora que o registro foi gerado pela orquestração" })
)

(
    trusted.write
    .option("overwriteSchema", "true")
    .format("delta")
    .mode("overwrite")
    .save("s3a://datalake/trusted/municipios_brasileiros")
)